### ProSite model PS51186

ProSite model PS51186: profile model of 718 proteins

Downloaded from: https://prosite.expasy.org/cgi-bin/aligner?psa=PS51186&format=FASTA

Information at https://prosite.expasy.org/PS51186

In [ ]:
head PS51186_in.tsv

In [ ]:
head -1 PS51186_in.tsv > PS51186_in_Pseudomonas
grep Pseudomonas PS51186_in.tsv >> PS51186_in_Pseudomonas
cat PS51186_in_Pseudomonas

In [ ]:
head -1 PS51186_in.tsv > PS51186_in_phage
grep 'Pseudomonas phage' PS51186_in.tsv >> PS51186_in_phage
cat PS51186_in_phage

### Constructing HMM

In [ ]:
cds_db="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/PaePhages_CDSs.fasta"

In [ ]:
hmmbuild PS51186.hmm PS51186.msa

In [ ]:
cat PS51186.hmm

PaePhages_CDSs.fasta: All CDSs of all accepted ICTV phages at NCBI Nucleotide
obtained by Batch Entrez of PaePhagesIDs.txt exporting to CDS fasta file

hmmer'ing the Prosite GNAT HMM to this CDS database

In [ ]:
hmmsearch -E 0.001 -o hmmer2.out --tblout hmmer2.table.out PS51186.hmm $cds_db

In [ ]:
grep -cE '[!\?]' hmmer2.out
grep -c '!' hmmer2.out

49 hits, of which 47 are significant

In [ ]:
cat hmmer2.table.out

### Generating report files

In [ ]:
metadata="/mnt/DATA/School/2022-2023/Thesis/Scripting/Mining/PaePhageInfoRetrieval/ICTV_Metadata_raw.txt"

Scraping the E-value, score and bias together with the labels, protein names and locations, and joining them to the species name

In [ ]:
echo -e "description of target" > descriptions
grep -vE '^#' hmmer2.table.out | \
    cut -c 159- | \
    grep -oE '\[protein=[A-Z|a-z|0-9|\/| |-]+\]' | \
    tr -d '[]' | \
    cut -c 9- >> descriptions
echo -e "\tlocation" > locations
grep -vE '^#' hmmer2.table.out | \
    cut -c 159- | \
    grep -oE '\[location=[complement(]*[0-9|\.|)]+\]' | \
    tr -d '[]' | \
    cut -c 9- | \
    tr '=' '\t' >> locations
echo -e "# target name\taccession\tquery name\taccession\tE-value\tscore\tbias\tE-value\tscore\tbias\texp\treg\tclu\tov\tenv\tdom\trep\tinc\t" > hmmer2.table.out.2
grep -vE '^#' hmmer2.table.out | cut -c -158 | tr -s ' ' '\t' >> hmmer2.table.out.2
paste -d '' hmmer2.table.out.2 descriptions locations | cut -f 1,5-7,19,20 > phmmer.out
rm descriptions
rm locations
rm hmmer2.table.out.2
cat phmmer.out

In [ ]:
accNr=$(cut -f 1 phmmer.out | grep -oE '\|.*\.[0-9]_p' | grep -oE '\|.*\.' | tr -d '|.')
echo "species accession" > accession
echo "target species" > species
echo "target genus" > genus
for nr in $accNr
do
grep $nr $metadata | cut -f 10 >> species
grep $nr $metadata | cut -f 8 >> genus
echo $nr >> accession
done
paste phmmer.out species accession genus > phmmer.out.species
rm species
rm accession
rm genus
cat phmmer.out.species

In [ ]:
cut -f 8,7,9 phmmer.out.species | tail -n +2 | sort -u | wc -l
cut -f 8,7,9 phmmer.out.species | tail -n +2 | wc -l

47 proteins distributed over 29 species

In [ ]:
cut -f 7-9 phmmer.out.species | tail -n +2 | sort | uniq > counted.species
cut -f 7-9 phmmer.out.species | tail -n +2 | sort | uniq -c | tr -s ' ' '\t' | cut -f 2 > counts.raw
paste counted.species counts.raw > counts
rm counted.species
rm counts.raw
cat counts

In [ ]:
cut -f 3 counts | sort | uniq -c

In [ ]:
wc -l counts